# #imports

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import psycopg2
from datetime import datetime, timedelta

In [2]:
conn_params = {
    "host" : "srvdados",
    "database" : "postgres",
    "user":"compras",
    "password": "pecist@compr@s2024"
}

In [3]:
query = """
    select 
        codpro, codfor,
        num_fab
    from"D-1".produto
"""

In [4]:
query2 = """
SELECT 
    pp.cod_pro,
    pp.dt_emissao,
    pp.qtde_ven AS vendas
FROM "D-1".prod_ped pp
JOIN "D-1".cliente c 
    ON pp.codcli = c.codcli
WHERE 
    pp.tipped = 'V' 
    AND pp.dt_emissao >= CURRENT_DATE - INTERVAL '2 years'
    AND c.codcli NOT IN ('13996','16100','18400','20000','23000','02608','24000','00270','20690','20691','20692','20693','23011','99999','88888','21097')
    AND pp.codvde NOT IN ('0100','0001','0006','2319')
    AND c.codarea <> '112'
    AND c.codcid <> '0501'
    AND pp.cd_loja = '08';
"""

In [5]:
query3 = """
    select 
        codpro,  
        sum(estoque) as estoque
    from "D-1".prd_loja
    where cd_loja = '08'
    group by codpro
"""

In [6]:
query4 = """
select 
     cd_produto, min(dt_estoque) as min_dt_estoque
	 from "D-1".estoque
     where cd_loja = '08'
     group by cd_produto
"""

In [7]:
with psycopg2.connect(**conn_params) as conn:
    produtos = pd.read_sql(query, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_9012\3954374250.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produtos = pd.read_sql(query, conn)


In [8]:
with psycopg2.connect(**conn_params) as conn:
    vendasg = pd.read_sql(query2, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_9012\2458323066.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vendasg = pd.read_sql(query2, conn)


In [9]:
with psycopg2.connect(**conn_params) as conn:
    estoquesg = pd.read_sql(query3, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_9012\3330857583.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  estoquesg = pd.read_sql(query3, conn)


In [10]:
with psycopg2.connect(**conn_params) as conn:
    dt_estoqueg = pd.read_sql(query4, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_9012\2911047658.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dt_estoqueg = pd.read_sql(query4, conn)


# #indicadores 6 meses

In [11]:
um_ano_atras = datetime.today() - timedelta(days=365)
vendas_ultimo_anog = vendasg[vendasg['dt_emissao'] >= um_ano_atras]  #vendas dos último ano

In [12]:
produtos_vendidosg = vendas_ultimo_anog['cod_pro'].unique()

In [13]:
produtos_sem_vendag = produtos[~produtos['codpro'].isin(produtos_vendidosg)] #identificar produtos sem vendas

In [14]:
produtos_com_estoqueg = pd.merge(produtos_sem_vendag, estoquesg, left_on='codpro', right_on='codpro', how='left')
produtos_com_estoqueg = produtos_com_estoqueg[produtos_com_estoqueg['estoque'] > 0]  

In [15]:
produtos_com_estoqueg = pd.merge(produtos_com_estoqueg, dt_estoqueg, left_on='codpro', right_on='cd_produto', how='left')

In [16]:
produtos_antigosg = produtos_com_estoqueg[produtos_com_estoqueg['min_dt_estoque'] <= um_ano_atras]

In [17]:
vendasg["dt_emissao"] = pd.to_datetime(vendasg["dt_emissao"])

In [ ]:
quantidade_produtosg = produtos_antigosg.shape[0]
lista_produtosg = produtos_antigosg[['codpro', 'codfor', 'estoque', 'min_dt_estoque']]

print("Quantidade de produtos nessa situação:", quantidade_produtosg)
print(lista_produtosg)

Quantidade de produtos nessa situação: 0
Empty DataFrame
Columns: [codpro, estoque, min_dt_estoque]
Index: []


In [19]:
seismeses_atrasg = datetime.today() - timedelta(days=180)
vendas_6mesesg = vendasg[vendasg['dt_emissao'] >= seismeses_atrasg]

In [20]:
produtos_vendidos6g = vendas_6mesesg['cod_pro'].unique()

In [21]:
produtos_sem_venda6g = produtos[~produtos['codpro'].isin(produtos_vendidos6g)]

In [22]:
#Mantém somente os que têm estoque > 0.
produtos_com_estoque6g = pd.merge(produtos_sem_venda6g, estoquesg, left_on='codpro', right_on='codpro', how='left')
produtos_com_estoque6g = produtos_com_estoque6g[produtos_com_estoque6g['estoque'] > 0]

In [23]:
produtos_com_estoque6g = pd.merge(produtos_com_estoque6g, dt_estoqueg, left_on='codpro', right_on='cd_produto', how='left')

In [24]:
#Mantém os que têm estoque registrado há ≥ 6 meses (min é a data mínima de estoque).
produtos_antigos6g = produtos_com_estoque6g[produtos_com_estoque6g['min_dt_estoque'] <= seismeses_atrasg]

In [26]:
quantidade_produtos6 = produtos_antigos6g.shape[0]
lista_produtos6_goiania = produtos_antigos6g[['codpro','codfor', 'estoque', 'min_dt_estoque']]

print("Quantidade de produtos nessa situação:", quantidade_produtos6)
lista_produtos6_goiania

Quantidade de produtos nessa situação: 7874


,codpro,codfor,estoque,min_dt_estoque
0,000032,00150,1.0,2024-10-28
1,000049,00254,2.0,2024-10-29
2,000064,00230,1.0,2024-10-25
3,000072,00230,1.0,2024-10-25
5,000140,00124,1.0,2024-10-25
...,...,...,...,...
9608,096124,00419,1.0,2025-02-24
9612,096140,04342,561.0,2025-03-05
9622,096596,02530,3.0,2025-03-09
9629,097142,00572,1.0,2025-03-11


In [27]:
#lista_produtos.to_csv("produtos1ano_gyn.csv")
lista_produtos6_goiania.to_csv("produtos6meses_gyn.csv")